In [1]:
# defining functions
import json 
import math

def getmeasurementTimestamp(item):
    return int(item['measurementTimestamp'])

def getProcessingTimestamp(item):
    return int(item['processingTimestamp'])

def get_x_error(item): #the error in the data is the stdev of the sample, we compute the error of the estimation (the sample mean)
    return item['value']['averagecoordinate']['error']['coordinates'][0]/math.sqrt(item['value']['trackeeHistory']['nMeasurements'])

def get_y_error(item):
    return item['value']['averagecoordinate']['error']['coordinates'][1]/math.sqrt(item['value']['trackeeHistory']['nMeasurements'])

def get_fitted(item):
     return item['value']['trackeeHistory']['fitStatus']
    
def get_x_sample_error(item): 
    return item['value']['averagecoordinate']['error']['coordinates'][0]
def get_y_sample_error(item):
    return item['value']['averagecoordinate']['error']['coordinates'][1]

def get_probChi2(item):
    return item['value']['trackeeHistory']['probChi2'] 
def get_Chi2PerDof(item):
    return item['value']['trackeeHistory']['chi2PerDof'] 

def plotHistogramOfDictionary(dictionary, xlabel, ylabel, nbins):
    dictionaryList = []
    for address in dictionary.keys():
        dictionaryList.append(dictionary[address])
    dictArray = np.array(dictionaryList)
    plt.hist(dictArray, bins = nbins)
    plt.ylabel(ylabel)
    plt.xlabel(xlabel)
    axes = plt.gca()
    plt.show()


In [2]:
#reading fitted data from the concert day
for i in [0]:
    data = []
    with open("arena/2015-07-05.json") as f:
    #with open("arena/2015-07-05_raw.json") as f:
        data = f.readlines()

    json_lines = []
    mac_adresses = []
    
    for line in data:
        jsline = json.loads(line)
        json_lines.append(jsline)#now json_lines contains all lines of data
        mac_adresses.append(jsline["value"]["sourceMac"]) # mac_addresses is a list of address per line
        
#sorting by time
json_lines.sort(key = getmeasurementTimestamp) # now json_lines is sorted by time

In [16]:
#computation of error etc
minTime = getmeasurementTimestamp(json_lines[0])
maxTime = getmeasurementTimestamp(json_lines[len(json_lines) - 1])
print("minTime="+ str(minTime))
print("maxTime="+ str(maxTime))

timeMinutes = (maxTime - minTime)/1000/60

for i in [0]:
    average_x_error = 0
    average_y_error = 0
    count = 0
    good_lines = []
    number_of_addresses =  len(set(mac_adresses)) #number of mac adresses detected. Not people, because of MAC randomization
    mc_good = []
    delay = 0
    number_of_fitted = 0
    for jsline in json_lines:
        average_x_error += get_x_error(jsline)
        average_y_error += get_y_error(jsline)
        thisdelay = getProcessingTimestamp(jsline) - getmeasurementTimestamp(jsline)
        delay += thisdelay
        if (get_fitted(jsline) == 'FITTED'):
            number_of_fitted +=1
        if get_x_error(jsline) <= 2 and get_y_error(jsline) <= 2:             
            good_lines.append(jsline)
            mc_good.append(jsline["value"]["sourceMac"])

    number_of_lines = len(json_lines)
    number_of_addresses_good = len(set(mc_good)) # number of addresses where error < 2m 
    average_x_error /= number_of_lines
    average_y_error /= number_of_lines
    delay /= number_of_lines
    
    print ('time in minutes:' + str(timeMinutes)) 
    print ('number of lines: ' + str(number_of_lines))
    print ('average x error: ' + str(average_x_error))
    print ('average y error: ' + str(average_y_error))    
    print('number of addresses the whole day:' + str(number_of_addresses))     
    #print('number of estimations with stdev < 2m:' + str(number_of_addresses_good))    #this is not correct 
    print('average delay in ms:' + str(delay))
    print('number of fitted:' + str(number_of_fitted)) 

minTime=1436047367297
maxTime=1436133804311
time in minutes:1440.6169
number of lines: 772867
average x error: 3342.886813949892
average y error: 3697.2146526257307
number of addresses the whole day:30344
number of estimations with stdev < 2m:17736
average delay in ms:10034.32547644032
number of fitted:772867


In [5]:
#computing FirstTimeSeen and LastTimeSeen
concertFinishedTimestamp=minTime+1000*60*60*8#timestamp until 8 am
FirstTimeSeen=dict()
LastTimeSeen=dict()

for jsline in json_lines:
    address = jsline["value"]["sourceMac"]
    time =  getmeasurementTimestamp(jsline)
    if address in FirstTimeSeen:        
        if time < FirstTimeSeen[address]:
            FirstTimeSeen[address] = time
    else:
        FirstTimeSeen[address] = time
        
    if address in LastTimeSeen:         
        if time > LastTimeSeen[address]:
            LastTimeSeen[address] = time
    else:
        LastTimeSeen[address] = time 

In [6]:
#Computing dwell time, number of persistent addresses, and number of addresses visible in every second
DwellTime = dict()
DwellTimeDuringConcert = dict()
numberOfAdressesAtConcert=0
for address in LastTimeSeen.keys():    
    DwellTime[address] = int((LastTimeSeen[address] - FirstTimeSeen[address]) /1000/60) # in minutes
    if LastTimeSeen[address] < concertFinishedTimestamp:
        numberOfAdressesAtConcert += 1
        DwellTimeDuringConcert[address] = DwellTime[address]  
print('number of addresses detected during concert hours:')
print(numberOfAdressesAtConcert)
longTermAddresses=[]
numberOfAddresses = []
AddressesInSec = []
for jsline in json_lines:
    sec = int(math.floor((getmeasurementTimestamp(jsline)- minTime)/1000))
    #print(str(sec))
    address = jsline["value"]["sourceMac"]
    if DwellTime[address] > 10:#  i.e. >10 minutes 
        longTermAddresses.append(address)
        if len(AddressesInSec) <= sec:
            while len(AddressesInSec) <= sec:
                AddressesInSec.append([])                
            AddressesInSec[sec].append(address)                       
        else:
            if address not in AddressesInSec[sec]:                                
                AddressesInSec[sec].append(address)
for setje in AddressesInSec:
    numberOfAddresses.append(len(setje)) 
longTermCount = len(set(longTermAddresses))                
print("Long term persistent addresses (>10 min)= " + str(longTermCount))
average = 0            
maxN = 0
for addresses in AddressesInSec:            
    average += len(addresses) 
    maxN = max(len(addresses), maxN)
average /= len(AddressesInSec)   
print("averageNumberOfAddressesPerSecondAtConcert " + str(average))
print(maxN)


number of addresses detected during concert hours:
28847
Long term persistent addresses (>10 min)= 10589
averageNumberOfAddressesPerSecondAtConcert 4.615643582683542
36


In [37]:
plotHistogramOfDictionary(DwellTimeDuringConcert, "dwell time(minutes)", "number of addresses", 50)

In [23]:
#drawing how many addresses per second are visible (later there is another way to compute this)
import matplotlib.pyplot as plt
plt.plot(numberOfAddresses)
plt.ylabel('addresses present')
plt.xlabel('sec')
axes = plt.gca()
axes.set_xlim([0,33000])
axes.set_ylim([0,max(numberOfAddresses)])
plt.show()

In [31]:
#unnecessary computations, same as histogram
n_addresses_staying_this_much = []
maxDwellTime = max(DwellTimeDuringConcert.values())
print("maxDwellTime during the whole day=" + str(maxDwellTime) + " minutes")
for i in range(maxDwellTime + 1):   
    n_addresses_staying_this_much.append(0)
    
for address in DwellTimeDuringConcert.keys():  
        n_addresses_staying_this_much[DwellTime[address]] +=1
#for i in range(0,470):
   # print(str(n_addresses_staying_this_much[i]))    
n_addresses_staying_less_than_10min =  sum(n_addresses_staying_this_much[:10])

n_addresses_staying_more_than_10min = numberOfAdressesAtConcert - n_addresses_staying_less_than_10min


print("number of addresses staying less than 10 min during concert hours  = " + str(n_addresses_staying_less_than_10min))
print("number of addresses staying more than 10 min during concert hours  = " + str(n_addresses_staying_more_than_10min))
 

maxDwellTime during the whole day=470 minutes
number of addresses staying less than 10 min during concert hours  = 18438
number of addresses staying more than 10 min during concert hours  = 10409


In [34]:
#plotting dwell time distribution, unnecessary
import matplotlib.pyplot as plt
plt.plot(n_addresses_staying_this_much)
plt.ylabel('number of addresses 05 july')
plt.xlabel('dwell time in minutes')
axes = plt.gca()
#axes.set_xlim([0,470])
#axes.set_ylim([0,200])
#axes.set_ylim([0,max(n_addresses_staying_this_much)])
plt.show()

In [ ]:
106/8

In [38]:
Randomized = dict()
PersistentRandomized = dict()
count0 = 0
count1 = 0
for line in json_lines:
    address = line["value"]["sourceMac"]
    if line["value"]["trackeeHistory"]["localMac"] == 1 :
        count1 +=1
        Randomized[address] = 1
        if DwellTime[address] > 10: 
            if LastTimeSeen[address] < concertFinishedTimestamp:
                PersistentRandomized[address] = 1 
    else:
        count0 +=1
        Randomized[address] = 0
        if DwellTime[address] > 10: 
            if LastTimeSeen[address] < concertFinishedTimestamp:
                PersistentRandomized[address] = 0 

zeros=0
ones=0
zerosPersistent = 0
onesPersistent = 0
for key in Randomized.keys():
    if Randomized[key]==0:
        zeros +=1
    else:
        ones +=1
for key in PersistentRandomized.keys():
    if PersistentRandomized[key]==0:
        zerosPersistent +=1
    else:
        onesPersistent +=1        
print("total number of lines with localMac == 1: " + str(count1))
print("total number of lines with localMac == 0: " + str(count0))
print("total number of addresses with localMac == 1: " + str(ones))
print("total number of addresses with localMac == 0: " + str(zeros))
print("total number of persistent addresses with localMac == 1: " + str(onesPersistent))
print("total number of persistent addresses with localMac == 0: " + str(zerosPersistent))
      


total number of lines with localMac == 1: 60024
total number of lines with localMac == 0: 712843
total number of addresses with localMac == 1: 15554
total number of addresses with localMac == 0: 14790
total number of persistent addresses with localMac == 1: 483
total number of persistent addresses with localMac == 0: 9769


In [40]:
DwellTimeConcertRandomized = dict()
DwellTimeConcertNonRandomized = dict()
for key in DwellTimeDuringConcert.keys():
    if Randomized[key] == 0:
        DwellTimeConcertNonRandomized[key] = DwellTimeDuringConcert[key]
    else:
        DwellTimeConcertRandomized[key] = DwellTimeDuringConcert[key]

    

In [47]:
plotHistogramOfDictionary(DwellTimeConcertNonRandomized, "dwell time(minutes) of non-randomized addresses", "number of addresses", 50)

In [49]:
plotHistogramOfDictionary(DwellTimeConcertRandomized, "dwell time(minutes) of randomized addresses", "number of addresses",200)

In [10]:
#double-check:another approach to computing visible addresses per second
Visible = dict()
for line in json_lines:
    address = line["value"]["sourceMac"]
    timeSec = int(math.floor((getmeasurementTimestamp(line)- minTime)/1000))
    if timeSec in Visible.keys():        
        Visible[timeSec].append(address)
    else:
        Visible[timeSec] = []
        Visible[timeSec].append(address)

maxSec = max(Visible.keys())    
print(maxSec)
for sec in range(0,maxSec):
    if sec not in Visible.keys():
        Visible[sec] = []
NVisible = []
for sec in range(0,86437):
    NVisible.append(len(set(Visible[sec])))
    

86437


In [60]:
def ComputeVisiblePerSec(json_lines, localMac):
    Visible = dict()
    for line in json_lines:
        address = line["value"]["sourceMac"]
        timeSec = int(math.floor((getmeasurementTimestamp(line)- minTime)/1000))
        if timeSec in Visible.keys():  
            if Randomized[address] == localMac:
                Visible[timeSec].append(address)
        else:
            Visible[timeSec] = []
            if Randomized[address] == localMac:
                Visible[timeSec].append(address)
    maxSec = max(Visible.keys())        
    for sec in range(0,maxSec):
        if sec not in Visible.keys():
            Visible[sec] = []
    NVisible = []
    for sec in range(0,maxSec):
        NVisible.append(len(set(Visible[sec])))
    return NVisible

In [61]:
VisiblePerSecondNonRandomized = ComputeVisiblePerSec(json_lines, 0)
VisiblePerSecondRandomized = ComputeVisiblePerSec(json_lines, 1) 

In [69]:
#drawing how many addresses per second are visible
import matplotlib.pyplot as plt
plt.plot(NVisible)
plt.ylabel('visible addresses')
plt.xlabel('time(sec)')
axes = plt.gca()
axes.set_xlim([0,30000])
axes.set_ylim([0,max(NVisible)])
plt.show()

In [70]:
plt.plot(VisiblePerSecondNonRandomized)
plt.ylabel('visible non-randomized addresses')
plt.xlabel('time(sec)')
axes = plt.gca()
axes.set_xlim([0,30000])
axes.set_ylim([0,max(VisiblePerSecondNonRandomized)])
plt.show()

In [71]:
plt.plot(VisiblePerSecondRandomized)
plt.ylabel('visible randomized addresses')
plt.xlabel('time(sec)')
axes = plt.gca()
axes.set_xlim([0,30000])
axes.set_ylim([0,max(VisiblePerSecondRandomized)])
plt.show()

In [1]:
######################################################################## packet rate analysis
#reading from raw data
import fileinput
import json 
json_lines_raw = []
for line in fileinput.input(['arena/2015-07-05_raw.json']):
    jsline = json.loads(line)
    data = []
    data.append(jsline["value"]["sourceMac"])
    data.append(jsline["value"]["localMac"])
    data.append(jsline["measurementTimestamp"])
    data.append(jsline["value"]["droneId"])
    data.append(jsline["value"]["signal"])
    json_lines_raw.append(data)
print(json_lines_raw[0])
        
      

['32bf72d3-d707-4c37-b9b5-6318187de63f', 0, 1436047299132, '107', -68]


In [3]:
#finding the most overloaded drone (112)
numberOfMeasurementsForDrone = dict()
for line in json_lines_raw:
    droneId = line[3]
    if droneId not in numberOfMeasurementsForDrone.keys():
        numberOfMeasurementsForDrone[droneId] = 0
    numberOfMeasurementsForDrone[droneId] +=1    
maxMeasurements = 0
maxDroneId = 0
for droneId in numberOfMeasurementsForDrone.keys():
    if numberOfMeasurementsForDrone[droneId] > maxMeasurements:
        maxMeasurements = numberOfMeasurementsForDrone[droneId]
        maxDroneId = droneId
print(maxDroneId)
print(maxMeasurements)

112
2920472


In [4]:
#computing the timestamps for every address visible at drone 112
timestamps112 = dict()
for line in json_lines_raw:
    if line[3]=="112":
        address = line[0]
        if address not in timestamps112.keys():
            timestamps112[address] = []        
        timestamps112[address].append(line[2])
for address in timestamps112.keys():
    timestamps112[address].sort()    

In [5]:
#computing the timestamps for every address 
timestamps = dict()
localMac = dict()
for line in json_lines_raw:
    address = line[0]
    if address not in timestamps.keys():
        timestamps[address] = []        
    timestamps[address].append(line[2])
    if address not in localMac.keys():   
        localMac[address] = line[1]
        

In [6]:
#sorting the timestamps for every address
for address in timestamps.keys():
    timestamps[address].sort()

In [7]:
#computing the interarrival times for every address
delays = dict()
for address in timestamps.keys():
    if address not in delays.keys():
        delays[address] = []
    oldTimeStamp = 0        
    for timestamp in timestamps[address]:
        if oldTimeStamp > 0:
            delay = (timestamp - oldTimeStamp)
            delays[address].append(delay)
        oldTimeStamp = timestamp
                            

In [8]:
#computing the interarrival times for every address at drone 112
delays112 = dict()
for address in timestamps112.keys():
    if address not in delays112.keys():
        delays112[address] = []
    oldTimeStamp = 0        
    for timestamp in timestamps112[address]:
        if oldTimeStamp > 0:
            delay = (timestamp - oldTimeStamp)
            delays112[address].append(delay)
        oldTimeStamp = timestamp

In [9]:
print(json_lines_raw[30000])
print(len(delays))
address1 = '32bf72d3-d707-4c37-b9b5-6318187de63f'
address2 = '3be779a3-1e1a-4b79-8ad0-c555f5124e7c'
print(localMac[address1])
print(localMac[address2])
print(delays[address1])
print(delays[address2])

['3be779a3-1e1a-4b79-8ad0-c555f5124e7c', 0, 1436047321397, '107']
126629
0
0
[9, 2, 31, 641, 0, 91, 21895, 94, 10, 98, 248791, 123, 6, 0, 2, 2, 3, 39, 34615, 41, 86, 41, 1, 42, 7, 9036, 41, 41, 55, 298, 100, 510, 61, 9, 16, 35, 6937, 43, 128, 1, 2, 558, 334, 8892, 43, 3, 207, 6, 17770, 65, 4, 1, 1, 6, 772, 33133, 1, 3, 8, 25, 2, 15, 65854, 0, 2, 11, 22, 107721, 33, 8, 1, 0, 41, 6, 5, 1843, 69, 80, 81, 1, 1, 6, 11, 2966, 3, 37, 2, 5, 0, 7, 4729, 7015, 20884, 7, 30776, 88032, 16495, 2138, 83102, 7, 19998, 0, 10394, 22178, 10, 43590, 13, 54991, 92, 7, 1, 4, 62679, 47048, 98150, 44253, 36885, 6, 7338, 2917, 6, 5589, 729, 58099, 21177, 7, 12, 24508, 54235, 36018, 13821, 83711, 5, 45959, 581848, 547, 119, 18, 6197, 22, 4709, 17, 38976, 8, 43, 31, 2, 3, 2, 35, 2, 2, 3, 0, 80483, 72, 2, 1, 4, 13, 3, 2, 2, 8571, 5, 2, 15, 13, 122, 718, 98857, 24, 42887, 31, 9, 32, 2, 1, 6, 34, 8265, 1, 2, 17, 1, 1, 17, 2, 1, 0, 6, 65, 18054, 20, 1, 2, 2, 3, 16, 2, 3, 86152, 42332, 234440, 44, 63605, 13, 24362, 

In [10]:
#computing statistics of all addresses
import numpy as np
averageDelay= dict()
medianDelay = dict()
stderrDelay = dict()
for address in delays.keys():
    if address not in averageDelay.keys():
        if len(delays[address]) > 1:
            delaysArray = np.array(delays[address])
            averageDelay[address] = np.mean(delaysArray)
            medianDelay[address] = np.median(delaysArray)
            stderrDelay[address] = np.std(delaysArray)
            

In [11]:
#computing statistics of non-randomized addresses
averageDelay0= dict()
medianDelay0 = dict()
stderrDelay0 = dict()
for address in delays.keys():
    if localMac[address] == 0:
        if address not in averageDelay0.keys():
            if len(delays[address]) > 1:
                delaysArray = np.array(delays[address])
                averageDelay0[address] = np.mean(delaysArray)
                medianDelay0[address] = np.median(delaysArray)
                stderrDelay0[address] = np.std(delaysArray)
               

In [12]:
#computing statistics of randomized addresses
averageDelay1= dict()
medianDelay1 = dict()
stderrDelay1 = dict()
for address in delays.keys():
    if localMac[address] == 1:
        if address not in averageDelay0.keys():
            if len(delays[address]) > 1:
                delaysArray = np.array(delays[address])
                averageDelay1[address] = np.mean(delaysArray)
                medianDelay1[address] = np.median(delaysArray)
                stderrDelay1[address] = np.std(delaysArray)

In [16]:
#computing statistics of non-randomized addresses for drone 112 (most overloaded drone)
averageDelay0_112= dict()
medianDelay0_112 = dict()
stderrDelay0_112 = dict()
for address in delays112.keys():
    if localMac[address] == 0:
        if address not in averageDelay0_112.keys():
            if len(delays112[address]) > 1:
                delaysArray = np.array(delays112[address])
                averageDelay0_112[address] = np.mean(delaysArray)
                medianDelay0_112[address] = np.median(delaysArray)
                stderrDelay0_112[address] = np.std(delaysArray)
                if len(delays112[address]) > 200:
                    print(address)
                if len(delays112[address])> 1000:    
                    print("long " +  address)
print('**********************************************************************')                
#computing statistics of randomized addresses for drone 112
averageDelay1_112= dict()
medianDelay1_112 = dict()
stderrDelay1_112 = dict()
for address in delays112.keys():
    if localMac[address] == 1:
        if address not in averageDelay0_112.keys():
            if len(delays112[address]) > 1:
                delaysArray = np.array(delays112[address])
                averageDelay1_112[address] = np.mean(delaysArray)
                medianDelay1_112[address] = np.median(delaysArray)
                stderrDelay1_112[address] = np.std(delaysArray)       
                if len(delays112[address]) > 200:
                    print(address)
                if len(delays112[address])> 1000:    
                    print("long " +  address)    

f7fad938-b05c-4950-be08-819cf858789c
10e2d965-5f9c-4065-aa3f-3435150ad7c9
cd927bfb-1dbf-4c7a-aa67-085e37ac643e
cd3b5f78-5111-41e7-8304-d68d68debb77
f6ddf914-61ae-4ba3-a97b-81ae14b85a1a
6368d5e2-8613-438e-a3a4-bd7927b6ca9e
03529407-cd5a-40cf-b86c-63f5b1dff8b7
b127b811-eb23-4550-876e-acf5b4362389
0ec3e4db-57f1-4736-a68e-0ec8648703cd
e0e8740a-b4ec-4df7-a9b3-c077d328f560
a5b8b115-6edf-414b-9b07-284b01e40c2c
c687fd41-a915-44d7-a77e-fc6674a69edd
49b22493-8890-4c29-85d0-080c1bc039be
3620f07e-a673-4648-bd9b-a0681d574d6f
long 3620f07e-a673-4648-bd9b-a0681d574d6f
198667d8-dbf7-4d05-a89c-ede661325aff
f70cec8b-8162-461c-a8b2-978bc0289413
65a5110d-f659-4219-ae05-b2aa95680460
54128958-8eec-445a-b9f3-fd60e6b1436d
1291e717-bb10-4ccf-9faf-2709a76f3a16
ed7ae39e-2b8e-4586-9399-1f2048d0e048
da5573c9-c4a8-4b81-ab52-cd5fab8daf31
b3293878-9db5-470c-812a-54d3ac913bc8
long b3293878-9db5-470c-812a-54d3ac913bc8
32f1f2aa-8358-49b8-8086-4bef5ba8b1a4
52f844c7-311c-46f3-a591-6689587b5ff4
2f1fa3db-21b5-4997-af47-f570

In [17]:
#drawing the delays of some non-randomized address
import matplotlib.pyplot as plt
plt.plot(delays112['62042a84-f904-4b2c-b79b-1e06c1ac980e'])
plt.ylabel('interarrival time (ms)')
plt.xlabel('packet')
#axes.set_ylim([0,200])
axes = plt.gca()

plt.show()

In [18]:

#drawing the delays of some randomized address
import matplotlib.pyplot as plt
plt.plot(delays112['7a43a795-b538-4563-b702-6f0256588479'])
plt.ylabel('interarrival time (ms)')
plt.xlabel('packet')
#axes.set_ylim([0,200])
axes = plt.gca()
print(delays112['7a43a795-b538-4563-b702-6f0256588479'])
plt.show()

[800, 44199, 31, 44966, 23, 721, 44255, 25, 19, 679, 44276, 23, 462, 44512, 737, 44261, 23, 447, 214, 44314, 24, 666]


In [19]:
print(delays112['443200e5-5752-4e5e-92f4-00af19a293be'])#a randomized address


[144882, 1390, 109, 1907, 453, 116, 562, 133927, 1666, 5170, 1128, 1028, 228, 111, 188, 374, 150, 522, 335, 714, 1534, 4045, 779, 901, 558, 905, 105, 1569, 92, 1263, 1229, 121, 1000, 450, 125, 1607, 402, 339, 328, 234, 230, 574279, 18344, 179882, 100, 1064, 3343, 680, 165, 279, 1690, 1008, 1122, 106, 347, 109, 82, 483, 446, 449, 3594, 1007, 3261, 170, 523, 204, 3466, 2611, 9, 1894, 1792, 1017, 551, 25915, 1048, 20, 503, 402, 2218, 935, 1685, 2038, 1336, 1340, 3036, 4160, 130, 19, 1046, 959, 2129, 449, 560, 97, 10, 1023, 1345, 113, 113, 17, 432, 2133, 1119, 3, 788, 677, 343, 213, 2179416, 226, 209, 685, 3027, 5731, 3469, 348, 565, 1124, 60, 724, 736, 48, 8, 56, 1194, 93, 562, 112, 669, 2025, 1911, 678, 103, 3040, 422, 17451, 903, 2698, 336, 114, 6852, 1117, 1014, 448, 451, 673, 682, 779, 2459, 579, 783, 794, 113, 107, 576, 208, 842, 395, 104, 25, 523, 14]


In [20]:
print(delays112['8b8a2356-d11e-4bd5-bb35-d8370bf48b1e'])# a randomized address

[160, 104, 528, 791, 1583, 264, 528, 264, 1056, 11, 780, 265, 528, 792, 265, 527, 445, 347, 264, 1584, 265, 1055, 791, 263, 529, 47, 1010, 560, 231, 264, 264, 263, 529, 264, 264, 528, 798, 192, 330, 2346, 358, 264, 792, 145, 914, 525, 530, 263, 528, 793, 64, 200, 265, 792, 792, 264, 264, 264, 264, 792, 528, 527, 264, 264, 2112, 527, 638, 417, 264, 528, 264, 528, 264, 264, 1847, 792, 1913, 263, 264, 528, 1056, 184, 80, 264, 527, 1319, 529, 263, 265, 527, 266, 262, 527, 264, 43, 221, 528, 264, 528, 264, 1056, 263, 792, 264, 527, 433, 95, 264, 263, 527, 265, 527, 528, 264, 792, 1319, 264, 30, 1025, 263, 528, 2706, 264, 263, 529, 263, 528, 264, 264, 528, 264, 130, 134, 1585, 264, 264, 264, 264, 1319, 529, 792, 529, 264, 4487, 11944, 528, 264, 790, 1695, 679, 1849, 791, 1257, 2440, 793, 2111, 1056, 1915, 1319, 1055, 2320, 583, 2640, 528, 792, 793, 264, 528, 1582, 264, 2112, 264, 528, 265, 264, 528, 792, 264, 264, 663, 3097, 160392, 264, 1056, 264, 1319, 7981, 264, 725, 595, 792, 528, 1319, 

In [21]:
#drawing the delays of some randomized address (the previous)
import matplotlib.pyplot as plt
plt.plot(delays112['8b8a2356-d11e-4bd5-bb35-d8370bf48b1e'])
plt.ylabel('interarrival time (ms)')
plt.xlabel('packet')
#axes.set_ylim([0,200])
axes = plt.gca()
plt.show()

In [22]:
arr = np.array(delays['32bf72d3-d707-4c37-b9b5-6318187de63f'])

In [23]:
plt.hist(arr, bins = 500)
plt.ylabel('frequency')
plt.xlabel('delay')
axes.set_xlim([0,200000])

axes = plt.gca()

plt.show()

In [24]:
#plotting histogram of average delay for non-randomized
averagedelaysList = []
for address in averageDelay0.keys():
    averagedelaysList.append(averageDelay[address])
averagedelays = np.array(averagedelaysList)
plt.hist(averagedelays, bins = 2000)
plt.ylabel('frequency')
plt.xlabel('average packet delay for non-randomized addresses')

#axes.set_xlim([0,200000])

axes = plt.gca()

plt.show()

In [49]:
#plotting histogram of average delay for randomized
averagedelaysList = []
for address in averageDelay1.keys():
    averagedelaysList.append(averageDelay[address])
averagedelays = np.array(averagedelaysList)
plt.hist(averagedelays, bins = 2000)
plt.ylabel('frequency')
plt.xlabel('average packet delay for randomized addresses')

#axes.set_xlim([0,200000])

axes = plt.gca()

plt.show()

In [91]:
plotHistogramOfDictionary(averageDelay0, 'freq', 'average packet delay for non-randomized addresses ',2000 )

In [ ]:
plotHistogramOfDictionary(medianDelay0_112, 'freq', 'median packet delay for non-randomized addresses for dron 112',2000 )

In [92]:
plotHistogramOfDictionary(stderrDelay0_112, 'freq', 'stdev packet delay for non-randomized addresses for dron 112',2000 )

In [11]:
# checking the trajectory of a certain address, error used is sample error (not error of the estimation of the mean) 
lines = []#will contain only the lines for that address
for line in json_lines:
    address = line["value"]["sourceMac"]
    #if (address == '27e573c8-1640-4ea8-86d8-0733c800e9cd'):#this is the address we are checking for, non-randomized
    if (address == '3620f07e-a673-4648-bd9b-a0681d574d6f'):
    #if (address == '8b8a2356-d11e-4bd5-bb35-d8370bf48b1e'):#randomized address    
        lines.append(line)
       
x_coord = []
y_coord = []
times = []
x_errors = []
y_errors = []
for line in lines:  
    alpha = get_probChi2(line)    
    if ( alpha > 0.4) and (get_x_sample_error(line) <10 ) and  get_Chi2PerDof(line) <1.1 :    
        coordinates = line["value"]["averagecoordinate"]["avg"]["coordinates"]
        time = math.floor(line["measurementTimestamp"]/1000)      
        if time not in times:       
            x_coord.append( coordinates[0])
            y_coord.append(coordinates[1])
            x_errors.append(get_x_sample_error(line)*2)# for a 95% confidence
            y_errors.append(get_y_sample_error(line)*2)# for a 95% confidence
            times.append(time)
    
print(len(x_errors)) 
print(len(x_coord))

84
84


In [77]:
# checking the trajectory of a certain address, error used is error of the estimation of the mean
lines = []#will contain only the lines for that address
for line in json_lines:
    address = line["value"]["sourceMac"]
    #if (address == '27e573c8-1640-4ea8-86d8-0733c800e9cd'):#this is the address we are checking for, nonrandomized
    #if address =='e1deda99-163f-4b68-9ade-e1f05d070bf2' : # the largest one, probably from the organizers  
    if (address == '3620f07e-a673-4648-bd9b-a0681d574d6f'):
    #if (address == '8b8a2356-d11e-4bd5-bb35-d8370bf48b1e'):#randomized address  
        lines.append(line)
       
x_coord = []
y_coord = []
times = []
x_errors = []
y_errors = []
for line in lines:
    alpha = get_probChi2(line)
    if ( alpha > 0.3) and (get_x_error(line) <5 ) and  get_Chi2PerDof(line) <1.2 :    
        coordinates = line["value"]["averagecoordinate"]["avg"]["coordinates"]
        time = math.floor(line["measurementTimestamp"]/1000)        
        if time not in times:
            x_coord.append( coordinates[0])
            y_coord.append(coordinates[1])
            x_errors.append(get_x_error(line)*2)# for a 95% confidence
            y_errors.append(get_y_error(line)*2)# for a 95% confidence
            times.append(time)
    
print(len(x_errors)) 
print(len(x_coord))

90
90


In [78]:
import numpy as np
import matplotlib.pyplot as plt

fig, (ax0, ax1) = plt.subplots(nrows=2, sharex=True)
ax0.errorbar(times, x_coord, yerr=x_errors, fmt='o')
ax0.set_title('x-coordinate with error')
plt.xlabel('time(sec)')

ax1.errorbar(times, y_coord, yerr=y_errors, fmt='o')
ax1.set_title('y-coordinate with error')

plt.show()


In [4]:
coor = dict()
count =0
for x in x_coord:
    key = times[count]
    coor[key] = x
    count +=1
plotHistogramOfDictionary(coor,  'x_coord', 'freq', 200)    
    

NameError: name 'x_coord' is not defined

In [12]:
# line: ['32bf72d3-d707-4c37-b9b5-6318187de63f', 0, 1436047299132, '107', -68]
# line: [address, localmac, time, drone, signal]
def GetSignalStrengthPerDroneForAddress(address):
    signalsDictionary = dict()
    for line in json_lines_raw: 
        mac = line[0]
        time = line[2]
        drone = line[3]
        signal = line[4]
        if mac == address:            
            pair=[time, signal]
            if drone not in signalsDictionary.keys():
                signalsDictionary[drone] = []                
            signalsDictionary[drone].append(pair)
    return signalsDictionary
    


In [13]:
signalsDict = GetSignalStrengthPerDroneForAddress('e1deda99-163f-4b68-9ade-e1f05d070bf2')

In [14]:
print(len(signalsDict))

23


In [15]:
print(signalsDict.keys())

dict_keys(['109', '108', '116', '128', '111', '122', '119', '125', '106', '112', '117', '123', '107', '104', '127', '129', '101', '130', '102', '103', '124', '121', '126'])


In [16]:
print (signalsDict['112'])

[[1436047302165, -64], [1436047302139, -65], [1436047307976, -64], [1436047307853, -64], [1436047317232, -67], [1436047327862, -63], [1436047337864, -64], [1436047337869, -64], [1436047337984, -64], [1436047337861, -66], [1436047337868, -63], [1436047337983, -65], [1436047337985, -65], [1436047347147, -64], [1436047357877, -62], [1436047357876, -62], [1436047357994, -63], [1436047361911, -65], [1436047362307, -62], [1436047362278, -63], [1436047367870, -63], [1436047367882, -65], [1436047367883, -62], [1436047377416, -65], [1436047387878, -64], [1436047387879, -64], [1436047387879, -64], [1436047387997, -63], [1436047397877, -65], [1436047406936, -66], [1436047406934, -64], [1436047407542, -65], [1436047417883, -61], [1436047417886, -64], [1436047418116, -70], [1436047417885, -63], [1436047417886, -66], [1436047421936, -61], [1436047427888, -63], [1436047428778, -65], [1436047437845, -62], [1436047437853, -63], [1436047447900, -64], [1436047447897, -63], [1436047452626, -61], [14360474

In [20]:
import matplotlib.pyplot as plt
def drawSignals(drone):
    plt.plot(signalsDict[drone])
    plt.ylabel('signal')
    plt.xlabel('time')
    axes = plt.gca()
    plt.show()



In [21]:
drawSignals('112')

In [14]:
import numpy as np
import matplotlib.pyplot as plt

pairs = signalsDict['126']
xx = []
yy=[]
for pair in pairs:
    xx.append(pair[0])
    yy.append(pair[1])
fig, (ax0, ax1) = plt.subplots(nrows=2, sharex=True)
ax0.errorbar(xx, yy, yerr=0, fmt='o')
ax0.set_title('signal')
plt.xlabel('timestamp(ms)')

ax1.errorbar(xx, yy, yerr=0, fmt='o')
ax1.set_title('signal')

plt.show()

NameError: name 'signalsDict' is not defined

In [27]:
print(signalsDict.keys())

dict_keys(['109', '108', '116', '128', '111', '122', '119', '125', '106', '112', '117', '123', '107', '104', '127', '129', '101', '130', '102', '103', '124', '121', '126'])
